In [2]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import KSA_XL_BOMD
from seqm.seqm_functions.read_xyz import read_xyz
from torch.profiler import profile, record_function, ProfilerActivity

DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


In [5]:
%%time
torch.manual_seed(0)

files = ['GS_dp.xyz']
species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

charges = torch.tensor([2],dtype=torch.int64, device=device)
molecule = Molecule(const, seqm_parameters, coordinates, species, charges).to(device)

output={'molid':[0], 'thermo':1, 'dump':1, 'prefix':'KSA-XL-BOMD_GS_dp'}
xl_bomd_params={'k':6, 'max_rank':2, 'err_threshold':0.0, 'T_el':1500}

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
md = KSA_XL_BOMD(xl_bomd_params, damp=100.0,
                   seqm_parameters=seqm_parameters, Temp=300.0, timestep=0.2, output=output).to(device)
md.initialize_velocity(molecule, vel_com=True )

_ = md.run(molecule, 10, reuse_P=False, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   286.77   6.523863e+00 -5.382925e+00 1.140939e+00 || 
     2   282.86   6.434983e+00 -5.320241e+00 1.114741e+00 || 
     3   279.81   6.365636e+00 -5.215901e+00 1.149735e+00 || 
     4   273.65   6.225409e+00 -5.073444e+00 1.151965e+00 || 
     5   266.50   6.062900e+00 -4.903841e+00 1.159060e+00 || 
     6   259.21   5.896944e+00 -4.704382e+00 1.192562e+00 || 
     7   249.49   5.675779e+00 -4.487322e+00 1.188457e+00 || 
     8   239.40   5.446261e+00 -4.258287e+00 1.187974e+00 || 
     9   229.20   5.214242e+00 -4.021065e+00 1.193177e+00 || 
    10   219.47   4.992828e+00 -3.787767e+00 1.205060e+00 || 
CPU times: user 55.9 s, sys: 1.27 s, total: 57.1 s
Wall time: 3.77 s
